In [16]:
# import nbformat
# from nbconvert.preprocessors import ExecutePreprocessor

# ExecutePreprocessor?

In [1]:
import os.path

is_production = os.environ.get('ENV') == 'production'
home = os.environ.get('HOME')

print('is pro', is_production)

vectors_path = vectors_path = '/tmp/' if is_production else home + '/ml-data/fasttext/'

In [2]:
# if not os.path.isfile(vectors_path + 'cc.ru.300.bin'):
#     !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.bin.gz -P {vectors_path}
#     !gunzip {vectors_path + 'cc.ru.300.bin.gz'}
        
# if not os.path.isfile(vectors_path + 'cc.ru.300.vec'):
#     !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz -P {vectors_path}
#     !gunzip {vectors_path + 'cc.ru.300.vec.gz'}

In [3]:
if is_production:
    import pymongo
    import pandas as pd
    from pymongo import MongoClient

    mongo_uri = os.environ['MONGO_URI']
    client = MongoClient(mongo_uri)
    db = client.cubes
    collection = db.events
    data = pd.DataFrame(list(collection.find({ "$where": "this.tags && this.tags.length > 0" }, { 'title': 1, 'text': 1, 'tags': 1, '_id': 0 })))
    data.to_csv('./data.csv')
else:
    !wget https://raw.githubusercontent.com/happylolonly/eventsfree-ml/master/notebooks/tags/data/data.csv

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('./data.csv')
df = df.replace(r'\\n',' ', regex=True) 

df['tags'] = df['tags'].map(eval) 
df['tags'] = df['tags'].map(lambda x: set(x))

df.head()

,Unnamed: 0,tags,text,title
0,0,{выставка},<p>&#x41D;&#x438;&#x43A;&#x442;&#x43E; &#x43D;...,Письма из Венеции
1,1,"{фитнес, праздник}",🎄🎄🎄А Вы уже ощущаете праздник и новогоднее нас...,🎄🎄🎄А Вы уже ощущаете праздник и новогоднее на...
2,2,"{фитнес, кроссфит, тренировка}",🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 \n \nВ э...,🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 \n \nВ ...
3,3,"{фитнес, кроссфит , тренировка}",БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! \n...,БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! \...
4,4,"{вебинар, работа, германия}","🇩🇪 Хотите работать в Германии, но не владеете ...","🇩🇪 Хотите работать в Германии, но не владеете..."


Remove lines where only 1 uniqie tag

In [6]:
from collections import Counter

counter = Counter()
for tags in df['tags'].values:
    for t in tags:
        counter[t] += 1
print(counter)

arr = []
for i in counter:
    if counter[i] > 2:
        arr.append(i);
        
print('total count:', len(counter), '\n >2 tags:', len(arr))

Counter({'музыка': 56, 'митап': 34, 'вечеринка': 29, 'фитнес': 21, 'праздник': 19, 'тренировка': 18, 'дети': 18, 'обучение': 16, 'it': 16, 'ярмарка': 16, 'DJ': 10, 'выставка': 8, 'лекция': 8, 'english': 6, 'танцы': 6, 'театр': 6, 'открытый микрофон': 6, 'джаз': 5, 'гитара': 5, 'стартап': 5, 'стендап': 5, 'нетворкинг': 5, 'дизайн': 4, 'турнир': 4, 'тенис': 4, 'фильм': 4, 'кино': 4, 'инвестиции': 4, 'маркет': 4, 'программирование': 4, 'кроссфит': 3, 'trx': 3, 'форро': 3, 'семинар': 3, 'концерт': 3, 'конкурс': 3, 'фотография': 3, 'право': 3, 'вебинар': 2, 'работа': 2, 'германия': 2, 'животные': 2, 'сальса': 2, 'чемпионат': 2, 'день рождения': 2, 'презентация': 2, 'финтех': 2, 'блокчейн': 2, 'круглый стол': 2, 'agile': 2, 'тестирование': 2, 'мастер-класс': 2, 'кроссфит ': 1, '3223': 1, 'рок': 1, 'йога': 1, 'бачата': 1, 'итальянский': 1, 'поэзия': 1, 'экскурсия': 1, 'свободный микрофон': 1, 'белорусский язык': 1, 'курсы': 1, 'спектакль': 1, 'hackathon': 1, 'blockchain': 1, 'гимнастика': 1, 

## tags_white_list = set(arr)
df['tags'] = df['tags'].map(lambda x: set(x).intersection(tags_white_list))
df = df[df.tags.map(len) > 0]

df.shape

In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
from stop_words import get_stop_words
nltk.download('wordnet')
stemmer = SnowballStemmer('russian')

import json
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].apply(strip_tags)
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"[^а-яА-Яa-zA-Z]", " ")
    df[text_field] = df[text_field].str.replace(r"\s+", " ")
    
    df[text_field] = df[text_field].str.lower()
    
    stop_words = get_stop_words('russian')
    
    def stem (text):
        temp = []
        for word in text.split(" "):
            if len(word) > 2 and word not in stop_words:
                temp.append(stemmer.stem(word))
        return " ".join(temp)
        
            
    df[text_field] = df[text_field].apply(stem)
    
    return df

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/happylol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df = standardize_text(df, 'text')
df = standardize_text(df, 'title')

df.head()

,Unnamed: 0,tags,text,title
0,0,{выставка},никт оста равнодушн венец город тысяч лиц суще...,письм венец
1,1,"{фитнес, праздник}",ощуща праздник новогодн настроен декабр фитнес...,ощуща праздник новогодн настроен декабр
2,2,"{фитнес, кроссфит, тренировка}",poisonbox дар возможн суббот декабр состо откр...,poisonbox дар возможн суббот декабр состо откр...
3,3,"{фитнес, кроссфит , тренировка}",бесплатн открыт тренировк crossf приход проб д...,бесплатн открыт тренировк crossf приход проб д...
4,4,"{вебинар, работа, германия}",хот работа герман владеет достаточн количеств ...,хот работа герман владеет достаточн количеств ...


In [9]:
rows = []

for row in df.to_dict(orient='record'):
    for tag in row['tags']:
        row_copy = row.copy()
#         print(tag)
        if tag == 'лекция':
            tag = 1
        else:
            tag = 0
        row_copy['tag'] = tag
        
        rows.append(row_copy)
        
df = pd.DataFrame(rows)

df['fulltext'] = df[['text', 'title']].apply(lambda x: ' '.join(x), axis=1)

print(df.shape)
print(df)

(434, 6)
     Unnamed: 0  tag                             tags  \
0             0    0                       {выставка}   
1             1    0               {фитнес, праздник}   
2             1    0               {фитнес, праздник}   
3             2    0   {фитнес, кроссфит, тренировка}   
4             2    0   {фитнес, кроссфит, тренировка}   
5             2    0   {фитнес, кроссфит, тренировка}   
6             3    0  {фитнес, кроссфит , тренировка}   
7             3    0  {фитнес, кроссфит , тренировка}   
8             3    0  {фитнес, кроссфит , тренировка}   
9             4    0      {вебинар, работа, германия}   
10            4    0      {вебинар, работа, германия}   
11            4    0      {вебинар, работа, германия}   
12            5    0      {вебинар, работа, германия}   
13            5    0      {вебинар, работа, германия}   
14            5    0      {вебинар, работа, германия}   
15            6    0        {фитнес, trx, тренировка}   
16            6    0  

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, stratify=df['tag'].values)

print(np.unique(train['tag'].values, return_counts=True))
print(np.unique(test['tag'].values, return_counts=True))

(array([0, 1]), array([297,   6]))
(array([0, 1]), array([129,   2]))


In [15]:
# %%bash

# pip install -U -q git+https://github.com/facebookresearch/fastText.git

import fastText

FIELD = 'fulltext'
train.head()

,Unnamed: 0,tag,tags,text,title,fulltext
392,194,0,"{it, agile, atlassian, митап}",aug meetup minsk митап организатор polontech я...,aug meetup minsk,aug meetup minsk митап организатор polontech я...
282,146,0,"{фитнес, пилатес, тренировка}",пролистыва чита мил формат тренировк дресс код...,пролистыва чита мил формат тренировк дресс код...,пролистыва чита мил формат тренировк дресс код...
137,64,0,{english},our next topic that books ar better than paper...,our next topic that books ar better than paper...,our next topic that books ar better than paper...
79,33,0,"{фотография, выставка}",книжн выставочн пространств жна шаф представля...,вне,книжн выставочн пространств жна шаф представля...
279,144,0,"{митап, agile, hr}",хорош найм мотивац удержан обучен корпоративн ...,bubbles and motivation christmas meetup for ян...,хорош найм мотивац удержан обучен корпоративн ...


In [12]:
def save(df, path):
    with open(path, 'w+') as f:
        for _, row in df.iterrows():
            f.write('__label__{} {}\n'.format(row['tag'], row[FIELD]))

In [14]:
# save(df, '/tmp/train.data')
save(train, '/tmp/train.data')
print(train)

     Unnamed: 0  tag                                       tags  \
392         194    0              {it, agile, atlassian, митап}   
282         146    0              {фитнес, пилатес, тренировка}   
137          64    0                                  {english}   
79           33    0                     {фотография, выставка}   
279         144    0                         {митап, agile, hr}   
169          83    0                        {вечеринка, музыка}   
70           28    0                 {фитнес, йога, тренировка}   
153          72    0                                  {english}   
361         181    0  {it, javascript, программирование, митап}   
375         186    0                           {дети, праздник}   
209         107    0                                  {ярмарка}   
179          90    0                      {it, обучение, митап}   
316         163    1       {лекция, программирование, обучение}   
336         171    0                  {джаз, вечеринка, музыка

In [ ]:
%%time

if not is_production:
    print('Start train not production')
    model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors=vectors_path + 'cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)
    
#     model.quantize(qnorm=True, cutoff=100000)
#     model.save_model('../../server/ml/tags/model/tags_model_new')
#     print('Model updated')
    

In [ ]:
# labels, probs = model.predict(test[FIELD].tolist(), k=5)
# labels = [ll[0].replace('__label__', '') for ll in labels]

In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(test['tag'].values, labels))

## Production model

In [ ]:
if is_production:
    print('Start train production')
    %%time
    save(df, '/tmp/train.data')
    model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors=vectors_path + 'cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)


    model.quantize(qnorm=True, cutoff=100000)
    model.save_model('./tags_model')

In [ ]:
labels, probs = model.predict(test[FIELD].tolist(), k=1)

In [ ]:
mistakes = 0;
for i, item in enumerate(test['tags']):
    length = len(item)
    
    prediction = map(lambda x: x.replace('__label__', ''), labels[i][0:length:])
    prediction = set(prediction)
    
    if (item != prediction):
        print(item, prediction)
        mistakes += 1
        
print('Accuracy:', (test.shape[0] - mistakes) / test.shape[0])

In [ ]:
mistakes = 0;
for i, item in enumerate(test['tags']):
    length = len(item)
    
    prediction = map(lambda x: x.replace('__label__', ''), labels[i][0:1:])
#     print(labels)
#     print(prediction)
    
#     print(prediction in item)
    
    is_mis = False
    for value in prediction:
        if (value not in item):
            is_mis = True
#     prediction = set(prediction)

    if (is_mis):
        mistakes += 1
#         print(prediction)
    
#     if (item != prediction):
#         print(item, prediction)
#         mistakes += 1
        
print('Accuracy:', (test.shape[0] - mistakes) / test.shape[0])